In [1]:
###################################################################################
#
#   CampusReadAndWriteInpV1.ipynb
#
#   CampusReadAndWriteINPAppS1andS2
#   Derived from ReadAndWriteINPAppV1
#   March 10, 2023
#   The purpose of this nroutine is to read an INP file, revise it's vavlues
#   then write a new INP file.  This will facilitate using ArcMap created objects 
#   into SWMM.  One of the key aspects is that the coordinates for polygon
#   catch basins and conduits need to be higher precision than the SWMM GUI
#   provides as a default.  This version used 8 decimals places the default is 3.
#   Care has to be taken not to overwrite the input INP if it contains the revised values 
#   when using SWMM.
#   This uses the PySWMM API to process the INP file.  The hope is that this will be able to be run with 
#   ArcPy to convert map object into SWMM objects,  SWMM can also be run with the API.
#   Michael E. Kress, Ph. D.
#   College of Staten Island, CUNY
#   March 17, 2023
#   Code cleanup
#   Incorporate component for SubCatchment vertices 
#   March 19, 2023
#   Code cleanup and verification
#   Incorporate use of *.JGW for the map dimensions and Backdrop
#   March 30, 2023
###   Notes: 1. At this time the number of rows in inp.POLYGONS[ ' Each S'] should be equal to the
#           the number of rows in the point data file.  If it is larger, it will not crash
#           but there will be extra entries which need to be deleted.
#           If there are more rows than entries in inp.POLYGONS it will crash.  Some programming needs to be done that 
#           delets the extra rows in inp.POLYGONS and append additional rows if the rows in the data are larger than
#           then inp.POLYGONS.
#             2.  The code should be revised to allow for more than one or two subcatchments.  This will require some kind of 
#           generalizing the name of the subcatchmens.  Perhaps restrict the subcatchment names to be S1, S2, etc which is
#           the SWMM approach.  At this time the subcatchment names is hard coded.
#     March 31, 2023
#     cleaned up code with subcatchment processing including appending and deleting coordinates from list.
#
#     April 1 and 2, 2023
#     worked on the subcatchment section and added S4 subcatchment.
#     The add and delete component is not automated.  It requires editing the inp file.
#     The backdrop file name is also not automated it has to be edited into the inp file
#     Also the jgw file needs to be specified in this program.
#     Next steps might be to put the jpg file name in this routine near where the jgw is defined.
#     Consider putting a section near the top that defines the input data files, subcatment names, etc.
#
#     April 17, 2023
#     Revise the code for handeling a change in the BackDrop file including 
#     changing the dimensions of the map based on the *.jgw file and changing 
#     the file name of the BackDrop *.jpg file.
#     Also note that the number of pixels (rows,columns) has to be revised for the
#     revised BackDrop file which is obtained when the *.jpg map file is 
#     exported from ArcMap.
#
##########################################################

In [2]:
#
#  Get the SWMMapi objects and set up encoding
#
import os
import csv
from decimal import Decimal, ROUND_DOWN
#
from swmm_api.input_file import read_inp_file, SwmmInput, section_labels as sections
from swmm_api.input_file.sections import Outfall
from swmm_api.input_file.macros.gis import write_geo_package, gpkg_to_swmm

In [3]:
#   Set the precision of the GIS variables in the configuration file 
from swmm_api import CONFIG
print(CONFIG)
CONFIG['gis_decimals'] = 10
print(CONFIG)

{'encoding': 'utf-8', 'gis_decimals': 3, 'exe_path': None}
{'encoding': 'utf-8', 'gis_decimals': 10, 'exe_path': None}


In [4]:
#   Note: Read input INP file, the "r" is for raw encoding for the "\" special character 
#   eg: read_inp_file(r'C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\Campus\CampusInputFile.inp')
#
#    Set working directory
#
print(os.getcwd())
#WDirectory='C:\\Zhanyang\\SensorData\\CompoundFlooding\\SWMM\\PySWMM\\Campus\\Check'
WDirectory='C:\\Zhanyang\\SensorData\\CompoundFlooding\\SWMM\\PySWMM\\Campus\\All4_3'
os.chdir(WDirectory)
print(os.getcwd())

C:\Users\M Kress
C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\Campus\All4_3


In [5]:
#
#   Specify input file and working directory.
# 
#InputInpFile='CampusInputFile.inp'
#InputInpFile='Site_Drainage_ModelTmp.inp'
#InputInpFile='NorthEastAndWestSubCatchments.inp'
InputInpFile='AllCampus4_3V1a.inp'
OutputInpFile='Out'+InputInpFile
print(InputInpFile)
print(OutputInpFile)

AllCampus4_3V1a.inp
OutAllCampus4_3V1a.inp


In [6]:
#
#   Define subroutine for reading csv file with gis data
#
def GetCsvTable(FileName):
#
#   Read text files with Coordinates of Loop Road Stream Sensor
#
#   Define File Name, Open file, Read file header and data.
#
# initializing the titles and rows list
#
    print(' FileName\n ',FileName)
    print(os.getcwd())
    print(os.path.exists(FileName))
    fields = []
    Rows = []
# opening the CSV file
    with open(FileName, mode ='r')as file:
# reading the CSV file
        csvFile = csv.reader(file)
        fields = next(csvFile)
        print(' Fields \n', fields)
        print(' End fields \n')
        for row in csvFile:
            Rows.append(row)
    return Rows, csvFile.line_num
#
#   End GetCsvTable
######################
#
#   LoopRdStream
#
StreamSensorRows,LenStreamSensorRows=GetCsvTable("CampusStreamSensor.txt")
print('\n LoopRdStream StreamSensorRows \n')
print(StreamSensorRows,LenStreamSensorRows)
#
#   Rain Gauge 
#
RainGaugeRows,LenRainGaugwRows=GetCsvTable("CSIGateWay.txt")
print('Rain Gauge \n', RainGaugeRows,LenRainGaugwRows,' \n')
#
#   Subcatchments 
#
#SubcatchmentRows,LenSubcatchmentRows=GetCsvTable("SubcatchmentCampusComplete.txt")
#
#   Subcatchment Nort, S1
#
SubcatchmentS1Rows,LenSubcatchmentS1Rows=GetCsvTable("SubCatchNorth1Points.txt")
print('Subcatchments Number of rows=',LenSubcatchmentS1Rows,' \n')
for row in SubcatchmentS1Rows:
    print (row)
#
#   Subcatchment East, S2
#
SubcatchmentS2Rows,LenSubcatchmentS2Rows=GetCsvTable("SubCatchmentEastPoints.txt")
print('Subcatchments Number of rows=',LenSubcatchmentS2Rows,' \n')
for row in SubcatchmentS2Rows:
    print (row)
#
#   Subcatchment North West, S3
#
SubcatchmentS3Rows,LenSubcatchmentS3Rows=GetCsvTable("SubCatchPCACenterPts.txt")
print('Subcatchments Number of rows=',LenSubcatchmentS3Rows,' \n')
for row in SubcatchmentS3Rows:
    print (row)
#
#   Subcatchment North West, S4
#
SubcatchmentS4Rows,LenSubcatchmentS4Rows=GetCsvTable("S4LibraryEastPts.txt")
print('Subcatchments Number of rows=',LenSubcatchmentS4Rows,' \n')
for row in SubcatchmentS4Rows:
    print (row)    
#
#   Subcatchment North West, S5
#
SubcatchmentS5Rows,LenSubcatchmentS5Rows=GetCsvTable("S5LibraryWestPts.txt")
print('Subcatchments Number of rows=',LenSubcatchmentS5Rows,' \n')
for row in SubcatchmentS5Rows:
    print (row)
#
#   Subcatchment North West, S6
#
SubcatchmentS6Rows,LenSubcatchmentS6Rows=GetCsvTable("SubCatchNorthWestPts.txt")
print('Subcatchments Number of rows=',LenSubcatchmentS6Rows,' \n')
for row in SubcatchmentS6Rows:
    print (row)
#
#   Subcatchment North West, S7
#
SubcatchmentS7Rows,LenSubcatchmentS7Rows=GetCsvTable("S7GreatLawnPts.txt")
print('Subcatchments Number of rows=',LenSubcatchmentS7Rows,' \n')
for row in SubcatchmentS7Rows:
    print (row)
#
#   Subcatchment North West, S8
#
SubcatchmentS8Rows,LenSubcatchmentS8Rows=GetCsvTable("S8ChemestryWestPts.txt")
print('Subcatchments Number of rows=',LenSubcatchmentS8Rows,' \n')
for row in SubcatchmentS8Rows:
    print (row)
#
#   Subcatchment North West, S9
#
SubcatchmentS9Rows,LenSubcatchmentS9Rows=GetCsvTable("S9OMRDDPts.txt")
print('Subcatchments Number of rows=',LenSubcatchmentS9Rows,' \n')
for row in SubcatchmentS9Rows:
    print (row)
#
#   Subcatchment North West, S10
#
SubcatchmentS10Rows,LenSubcatchmentS10Rows=GetCsvTable("S10SouthWestPts.txt")
print('Subcatchments Number of rows=',LenSubcatchmentS10Rows,' \n')
for row in SubcatchmentS10Rows:
    print (row)
#
#   Subcatchment North West, S11
#
SubcatchmentS11Rows,LenSubcatchmentS11Rows=GetCsvTable("S11NorthWestPts.txt")
print('Subcatchments Number of rows=',LenSubcatchmentS11Rows,' \n')
for row in SubcatchmentS11Rows:
    print (row)
#
#   Save the tabels for assignment later in the program
#
SCRow=(SubcatchmentS1Rows,SubcatchmentS2Rows,SubcatchmentS3Rows,SubcatchmentS4Rows,SubcatchmentS5Rows,
      SubcatchmentS6Rows,SubcatchmentS7Rows,SubcatchmentS8Rows,SubcatchmentS9Rows,SubcatchmentS10Rows,
      SubcatchmentS11Rows)
LenSCRows=((LenSubcatchmentS1Rows-1), (LenSubcatchmentS2Rows-1),(LenSubcatchmentS3Rows-1),(LenSubcatchmentS4Rows-1),
          LenSubcatchmentS5Rows-1,LenSubcatchmentS6Rows-1,LenSubcatchmentS7Rows-1,LenSubcatchmentS8Rows-1,
          LenSubcatchmentS9Rows-1,LenSubcatchmentS10Rows-1,LenSubcatchmentS11Rows-1)
SCname=('S1','S2','S3','S4','S5','S6','S7','S8','S9','S10','S11')



 FileName
  CampusStreamSensor.txt
C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\Campus\All4_3
True
 Fields 
 ['FID', 'Name', 'POINT_X', 'POINT_Y', 'Gateway', 'Source']
 End fields 


 LoopRdStream StreamSensorRows 

[['0', 'LoopRd Stream', '-74.1534141443', '40.6069063208', 'CSI', 'Rain']] 2
 FileName
  CSIGateWay.txt
C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\Campus\All4_3
True
 Fields 
 ['FID', 'Id', 'Lon', 'Lat', 'Comment1', 'Comment2']
 End fields 

Rain Gauge 
 [['0', '1', ' -74.1486150000 ', ' 40.6041250000', ' ', ' ']] 2  

 FileName
  SubCatchNorth1Points.txt
C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\Campus\All4_3
True
 Fields 
 ['FID', 'Name', 'POINT_X', 'POINT_Y']
 End fields 

Subcatchments Number of rows= 18  

['0', 'S1', '-74.146246079199997', '40.606068270400002']
['1', 'S1 ', '-74.143795234400002', '40.605449610599997']
['2', 'S1 ', '-74.142986217599997', '40.605520994400003']
['3', 'S1', '-74.143152779900007', '40.603783987900002']
['4', 'S1',

In [7]:
#
#   Get Dimensions and locations of Backdrop from text file.
#   This section has been replaced with the use of the *.JGW file.
#
DimensionsRows,LenDimensionsRows=GetCsvTable("BackdropMap.txt")
for row in DimensionsRows:
    print (row)
print(' DimensionsRows[0][0] '+ str(DimensionsRows[0][0]))


 FileName
  BackdropMap.txt
C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\Campus\All4_3
True
 Fields 
 ['XMin', 'YMin', 'XMax', 'YMax ']
 End fields 

['-74.1695179217', '40.5839016561', '-74.1252302037', '40.6152011491']
 DimensionsRows[0][0] -74.1695179217


In [8]:
#
#   Read text file of floating point numbers.
#
def readFlot(InFile):
    file_in = open(InFile, "r")
    r  =  file_in.readlines()
    file_in.close()
    l = 0
    x = []
    while l < len(r) :
        flt = float(r[l])
        x.append(flt)
        l += 1
    return x, r


In [9]:
#
#   Read Backdrop world coordinate file.
#   This is an alternate way of getting the dimension and 
#   registrating the backdrop image.
#   This file has to be coordinated with the Backdrop Image file.
#   They do not have to be the same file name but it needs to have the 
#   same image registration values.
#
#WorldCoordinatesFile="CampusSWMMUtility3_30.jgw"
#WorldCoordinatesFile="CampusSWMMUtility4_2.jgw"
#BackDropFile="CampusSWMMUtility4_2.jpg"
WorldCoordinatesFile="CampusSWMMLandUse.jgw"
BackDropFile="CampusSWMMLandUse.jpg"
#   for CampusSWMMLandUse  get the sizes when exporting the map from arcMap
NColsWD=7755.00
NRowsWD=3604.00
#
#   Note:
#   The number of rows and columns corresponds to the raster of the image
#   in the *.JPG Background file.  I used arcMap to read the *.JPG file to get the characteristics of 
#   the raster image.  It may also be shown when the map image is exported from arcMap.
#
#  for CampusSWMMUtility3_12 and 3_10
#NRowsWD=3740.0
#NColsWD=7755.0
#   for CampusSWMMUtility3_30  get the sizes when exporting the map from arcMap
#NRowsWD=3740.0
#NColsWD=7641.0
#   for CampusSWMMUtility4_2  get the sizes when exporting the map from arcMap
#NColsWD=7641.00
#NRowsWD=3604.00

#
InFile=open(WorldCoordinatesFile,"r")
print("readlines function:")
WDR=InFile.readlines()
InFile.close()
print(WDR)
print(WorldCoordinatesFile +' read')
#
#   Convert string variable to floats.
#
WDRF=[]
WDRF[:]=(float(WDR[0]),float(WDR[1]),float(WDR[2]),float(WDR[3]),float(WDR[4]),float(WDR[5]))
print(WDRF)
#   Initialize the window variables and set their precision.
XMinWD=0.000000000000
YMinWD=0.000000000000
XMaxWD=0.000000000000
YMaxWD=0.000000000000
#
#   Set the Map window dimensions based of the *.jpg file.
#   This formula is available at: https://www.openswmm.org/Topic/3136/swmm5-back-drop-image-world-reference
#    
XMinWD=WDRF[4]
YMinWD=WDRF[5]+WDRF[3]*NRowsWD
XMaxWD=XMinWD+WDRF[0]*NColsWD
YMaxWD=WDRF[5]
#
print('XMinWD, YMinWD, XMaxWD, YMaxWD ' + str(XMinWD)+','+str(YMinWD)+','+str(XMaxWD)+','+str(YMaxWD))

readlines function:
['0.000005710859829\n', '0.000000000000000\n', '0.000000000000000\n', '-0.000005710970505\n', '-74.171350445584977\n', '40.609655632391835\n']
CampusSWMMLandUse.jgw read
[5.710859829e-06, 0.0, 0.0, -5.710970505e-06, -74.17135044558498, 40.609655632391835]
XMinWD, YMinWD, XMaxWD, YMaxWD -74.17135044558498,40.589073294691815,-74.12706272761108,40.609655632391835


In [10]:
#
#  Check directory and valid file name.
#
print(os.getcwd())
print(os.path.exists(InputInpFile))
#
#   Read the inp file and print it to verify.
#
inp = read_inp_file(InputInpFile)
print(inp)


C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\Campus\All4_3
True
{'TITLE': ';;Project Title/Notes\nCSI Campus\n\n;;____________________________________________________________________________________________________', 'OPTIONS': {   'FLOW_UNITS': 'CFS',
    'INFILTRATION': 'GREEN_AMPT',
    'FLOW_ROUTING': 'KINWAVE',
    'LINK_OFFSETS': 'DEPTH',
    'MIN_SLOPE': 0,
    'ALLOW_PONDING': True,
    'SKIP_STEADY_STATE': False,
    'START_DATE': datetime.date(2023, 1, 22),
    'START_TIME': datetime.time(0, 0),
    'REPORT_START_DATE': datetime.date(2023, 1, 22),
    'REPORT_START_TIME': datetime.time(0, 0),
    'END_DATE': datetime.date(2023, 1, 26),
    'END_TIME': datetime.time(23, 59),
    'SWEEP_START': '01/01',
    'SWEEP_END': '12/31',
    'DRY_DAYS': 1,
    'REPORT_STEP': datetime.time(0, 5),
    'WET_STEP': datetime.time(0, 5),
    'DRY_STEP': datetime.time(1, 0),
    'ROUTING_STEP': '0:00:20',
    'RULE_STEP': datetime.time(0, 0),
    'INERTIAL_DAMPING': 'PARTIAL',
    'NORM

In [11]:
print(inp)
print(inp[sections.POLYGONS].to_inp_lines())

{'TITLE': ';;Project Title/Notes\nCSI Campus\n\n;;____________________________________________________________________________________________________', 'OPTIONS': {   'FLOW_UNITS': 'CFS',
    'INFILTRATION': 'GREEN_AMPT',
    'FLOW_ROUTING': 'KINWAVE',
    'LINK_OFFSETS': 'DEPTH',
    'MIN_SLOPE': 0,
    'ALLOW_PONDING': True,
    'SKIP_STEADY_STATE': False,
    'START_DATE': datetime.date(2023, 1, 22),
    'START_TIME': datetime.time(0, 0),
    'REPORT_START_DATE': datetime.date(2023, 1, 22),
    'REPORT_START_TIME': datetime.time(0, 0),
    'END_DATE': datetime.date(2023, 1, 26),
    'END_TIME': datetime.time(23, 59),
    'SWEEP_START': '01/01',
    'SWEEP_END': '12/31',
    'DRY_DAYS': 1,
    'REPORT_STEP': datetime.time(0, 5),
    'WET_STEP': datetime.time(0, 5),
    'DRY_STEP': datetime.time(1, 0),
    'ROUTING_STEP': '0:00:20',
    'RULE_STEP': datetime.time(0, 0),
    'INERTIAL_DAMPING': 'PARTIAL',
    'NORMAL_FLOW_LIMITED': 'BOTH',
    'FORCE_MAIN_EQUATION': 'H-W',
    'VARIAB

In [12]:
#######
#
#   subroutine to process the vertices of a subcatchment polygon vertices which includes:
#   1.  Read a text file including the updated vertices generated in arcMap
#   2.  Compare the number of rows of data in the text file table, LenSubcatchmentRows to the number of 
#       vertices in the list of polygons from the inp file, lengthVertices.
#   3.  There are 3 cases to handle: 
#         i.  lengthVertices==LenSubcatchmentRows
#         ii. lengthVertices>LenSubcatchmentRows  
#                   assign values for LenSubcatchmentRows and delete the additional items
#         iii. lengthVertices<LenSubcatchmentRows  
#                   assign values for LenSubcatchmentRows and append the additional items to the inp variables.
#
########
#   LenSCRows, SCnames and SCRows were assigned earlier in the program.
#LenSCRows=((LenSubcatchmentS1Rows-1), (LenSubcatchmentS2Rows-1), (LenSubcatchmentS3Rows-1))
#SCname=('S1','S2','S3')
#SCRow=(SubcatchmentS1Rows,SubcatchmentS2Rows,SubcatchmentS3Rows)
LVertices=[]
for ir in range(len(SCname)):
    lengthVertices = len(inp.POLYGONS[SCname[ir]].polygon)
    LVertices.append(lengthVertices)
    print(' Subcatchment '+str(SCname[ir])+' '+ str(lengthVertices) +' Number of rows '+str((LenSCRows[ir]))+'\n')
    #
    if LVertices[ir] == LenSCRows[ir]:
    #
    #   Number of rows equale number of vertices.
    #   Replace the vertices in inp with the values from the text file.
    #
        print('ir ',ir,' LV ',LVertices[ir], ' LSCR ',LenSCRows[ir])
        for irv in range(LenSCRows[ir]):
            print('ir,irv Name ',ir,irv,SCname[ir],'\n')
            inp.POLYGONS[SCname[ir]].polygon[irv][0]=Decimal(str(SCRow[ir][irv][2])).quantize(Decimal('.00000000001'))
            inp.POLYGONS[SCname[ir]].polygon[irv][1]=Decimal(str(SCRow[ir][irv][3])).quantize(Decimal('.00000000001'))
    elif LVertices[ir] < LenSCRows[ir]:
        print('ir ',ir,' LV ',LVertices[ir], ' LSCR ',LenSCRows[ir])
        print(' More rows than vertices, append additional rows')
        #
        #   Append additional rows from text file
        #
    elif LVertices[ir] > LenSCRows[ir]:
        print('ir ',ir,' LV ',LVertices[ir], ' LSCR ',LenSCRows[ir])
        print(' More vertices than rows, delete additional additional vertices')
        #
        #   delete extra vertices from polygon list
        #
#
#
#   Verify resulting vertice lists
#
for ir in range(len(SCname)): 
    print('Verify '+SCname[ir] +' SC \n')
    for irv in range(LenSCRows[ir]):
        print(str(inp.POLYGONS[SCname[ir]].polygon[irv][0])+' '+ str(inp.POLYGONS[SCname[ir]].polygon[irv][1]))
#
#   End Subcatchment polygon processing
#

 Subcatchment S1 17 Number of rows 17

ir  0  LV  17  LSCR  17
ir,irv Name  0 0 S1 

ir,irv Name  0 1 S1 

ir,irv Name  0 2 S1 

ir,irv Name  0 3 S1 

ir,irv Name  0 4 S1 

ir,irv Name  0 5 S1 

ir,irv Name  0 6 S1 

ir,irv Name  0 7 S1 

ir,irv Name  0 8 S1 

ir,irv Name  0 9 S1 

ir,irv Name  0 10 S1 

ir,irv Name  0 11 S1 

ir,irv Name  0 12 S1 

ir,irv Name  0 13 S1 

ir,irv Name  0 14 S1 

ir,irv Name  0 15 S1 

ir,irv Name  0 16 S1 

 Subcatchment S2 11 Number of rows 11

ir  1  LV  11  LSCR  11
ir,irv Name  1 0 S2 

ir,irv Name  1 1 S2 

ir,irv Name  1 2 S2 

ir,irv Name  1 3 S2 

ir,irv Name  1 4 S2 

ir,irv Name  1 5 S2 

ir,irv Name  1 6 S2 

ir,irv Name  1 7 S2 

ir,irv Name  1 8 S2 

ir,irv Name  1 9 S2 

ir,irv Name  1 10 S2 

 Subcatchment S3 9 Number of rows 9

ir  2  LV  9  LSCR  9
ir,irv Name  2 0 S3 

ir,irv Name  2 1 S3 

ir,irv Name  2 2 S3 

ir,irv Name  2 3 S3 

ir,irv Name  2 4 S3 

ir,irv Name  2 5 S3 

ir,irv Name  2 6 S3 

ir,irv Name  2 7 S3 

ir,irv Name  2 

In [13]:
#   Check the precision of the coordinates of the LoopRdStream.
#
print(inp[sections.COORDINATES].to_inp_lines())
#
#    Set the location with higher precision of the coordinates of the LoopRdStreem Outfall 
#
inp.COORDINATES['LoopRdStream'].x =-74.1534141443
inp.COORDINATES['LoopRdStream'].y = 40.6069063208
print(inp[sections.COORDINATES].to_inp_lines())



;;                    x         y
;;node                           
LoopRdStream  -74.15341  40.60691
Out1            -74.155    40.603
Out2            -74.155      40.6
Out3            -74.153    40.596
;;                    x         y
;;node                           
LoopRdStream  -74.15341  40.60691
Out1            -74.155    40.603
Out2            -74.155      40.6
Out3            -74.153    40.596


In [14]:

#
#   Assign coordinates to LoopRdStream 
#
print(type(StreamSensorRows[0][2]))
#print(Decimal(str(rows[0][2])).quantize(Decimal('.000000001'), rounding=ROUND_DOWN))
inp.COORDINATES['LoopRdStream'].x =Decimal(str(StreamSensorRows[0][2])).quantize(Decimal('.00000000001'), rounding=ROUND_DOWN)
inp.COORDINATES['LoopRdStream'].y =Decimal(str(StreamSensorRows[0][3])).quantize(Decimal('.00000000001'), rounding=ROUND_DOWN)
#
#   Check values of coordinates.
#
print(inp.COORDINATES['LoopRdStream'].x, ' x \n ')
print(inp.COORDINATES['LoopRdStream'].y, ' y \n ')


<class 'str'>
-74.15341414430  x 
 
40.60690632080  y 
 


In [15]:
print(inp[sections.SYMBOLS].to_inp_lines())
#
print(inp.SYMBOLS['CSI1'].x )
print(inp.SYMBOLS['CSI1'].y )

;;              x         y
;;gage                     
CSI1    -74.14862  40.60413
-74.148615
40.604125


In [16]:

#
#   assignment.
#
inp.SYMBOLS['CSI1'].x =Decimal(RainGaugeRows[0][2]).quantize(Decimal('.00000000001'))
inp.SYMBOLS['CSI1'].y =Decimal(RainGaugeRows[0][3]).quantize(Decimal('.00000000001'))
#inp.SYMBOLS['CSI1'].x = -74.148615000000007
#inp.SYMBOLS['CSI1'].y = 40.604125000000003
#   Verify assignment.
print(inp.SYMBOLS['CSI1'].x ,' \n ')
print(inp.SYMBOLS['CSI1'].y ,' \n ')

-74.14861500000  
 
40.60412500000  
 


In [17]:
#   Calculation from jgw file:
#-74.1695179216830	40.5888707163314	-74.1252273482792	40.6102263788570

#
print(inp.MAP)
print(inp.MAP['UNITS'])
print(inp.MAP['DIMENSIONS'][:])
#
#   Using jgw file
#
inp.MAP['DIMENSIONS'][0]=XMinWD
inp.MAP['DIMENSIONS'][1]=YMinWD
inp.MAP['DIMENSIONS'][2]=XMaxWD
inp.MAP['DIMENSIONS'][3]=YMaxWD
#
print(inp.MAP['DIMENSIONS'][:])

{'DIMENSIONS': [-74.16993, 40.58908, -74.1263, 40.60966], 'UNITS': 'None'}
None
[-74.16993, 40.58908, -74.1263, 40.60966]
[-74.17135044558498, 40.589073294691815, -74.12706272761108, 40.609655632391835]


In [18]:
#
#   Backdrop
#'BACKDROP': 'FILE       "CampusSWMMUtility3_12.jpg"\nDIMENSIONS -74.169542 40.588979 -74.125310 40.610348'
print('Backdrop file in *.inp file:')
print(inp.BACKDROP)
print(inp.BACKDROP['FILE'])
print(inp.BACKDROP['DIMENSIONS'])
#
#   Using jgw file
#
inp.BACKDROP['DIMENSIONS'][0]=XMinWD
inp.BACKDROP['DIMENSIONS'][1]=YMinWD
inp.BACKDROP['DIMENSIONS'][2]=XMaxWD
inp.BACKDROP['DIMENSIONS'][3]=YMaxWD
print('  Revised Backdrop Dimensions'+str(inp.BACKDROP['DIMENSIONS'][:]))
inp.BACKDROP['FILE']=BackDropFile
print('  Revised BackDrop file ',inp.BACKDROP['FILE'])

Backdrop file in *.inp file:
{'FILE': '"CampusSWMMUtility4_2.jpg"', 'DIMENSIONS': [-74.16993, 40.58908, -74.1263, 40.60966]}
"CampusSWMMUtility4_2.jpg"
[-74.16993, 40.58908, -74.1263, 40.60966]
  Revised Backdrop Dimensions[-74.17135044558498, 40.589073294691815, -74.12706272761108, 40.609655632391835]
  Revised BackDrop file  CampusSWMMLandUse.jpg


In [19]:

#
#  Save the current INP configuration into a file.
#
#   Note: r is for encoding \
#inp.write_file(r'C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\New_inputfile.inp')
inp.write_file(OutputInpFile)

'OutAllCampus4_3V1a.inp'

In [20]:
from swmm_api.input_file import read_inp_file, SwmmInput, section_labels as sections
from swmm_api.input_file.sections import Outfall

In [21]:
 #inp = SwmmInput.read_file(r'C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\Site_Drainage_Model.inp')
#   Read Output file to see if the precision is preserved.
inp1 = SwmmInput.read_file(OutputInpFile)

In [22]:
 inp1.OPTIONS 

{   'FLOW_UNITS': 'CFS',
    'INFILTRATION': 'GREEN_AMPT',
    'FLOW_ROUTING': 'KINWAVE',
    'LINK_OFFSETS': 'DEPTH',
    'MIN_SLOPE': 0,
    'ALLOW_PONDING': True,
    'SKIP_STEADY_STATE': False,
    'START_DATE': datetime.date(2023, 1, 22),
    'START_TIME': datetime.time(0, 0),
    'REPORT_START_DATE': datetime.date(2023, 1, 22),
    'REPORT_START_TIME': datetime.time(0, 0),
    'END_DATE': datetime.date(2023, 1, 26),
    'END_TIME': datetime.time(23, 59),
    'SWEEP_START': '01/01',
    'SWEEP_END': '12/31',
    'DRY_DAYS': 1,
    'REPORT_STEP': datetime.time(0, 5),
    'WET_STEP': datetime.time(0, 5),
    'DRY_STEP': datetime.time(1, 0),
    'ROUTING_STEP': '0:00:20',
    'RULE_STEP': datetime.time(0, 0),
    'INERTIAL_DAMPING': 'PARTIAL',
    'NORMAL_FLOW_LIMITED': 'BOTH',
    'FORCE_MAIN_EQUATION': 'H-W',
    'VARIABLE_STEP': 0.75,
    'LENGTHENING_STEP': 0,
    'MIN_SURFAREA': 12.566,
    'MAX_TRIALS': 8,
    'HEAD_TOLERANCE': 0.005,
    'SYS_FLOW_TOL': 5,
    'LAT_FLOW_TOL': 

In [23]:
 inp1.keys()

dict_keys(['TITLE', 'OPTIONS', 'EVAPORATION', 'RAINGAGES', 'SUBCATCHMENTS', 'SUBAREAS', 'INFILTRATION', 'OUTFALLS', 'REPORT', 'TAGS', 'MAP', 'COORDINATES', 'VERTICES', 'POLYGONS', 'SYMBOLS', 'LABELS', 'BACKDROP'])

In [24]:
 print(inp1[sections.OPTIONS].to_inp_lines())

FLOW_UNITS           CFS
INFILTRATION         GREEN_AMPT
FLOW_ROUTING         KINWAVE
LINK_OFFSETS         DEPTH
MIN_SLOPE            0
ALLOW_PONDING        YES
SKIP_STEADY_STATE    NO
START_DATE           01/22/2023
START_TIME           00:00:00
REPORT_START_DATE    01/22/2023
REPORT_START_TIME    00:00:00
END_DATE             01/26/2023
END_TIME             23:59:00
SWEEP_START          01/01
SWEEP_END            12/31
DRY_DAYS             1
REPORT_STEP          00:05:00
WET_STEP             00:05:00
DRY_STEP             01:00:00
ROUTING_STEP         0:00:20
RULE_STEP            00:00:00
INERTIAL_DAMPING     PARTIAL
NORMAL_FLOW_LIMITED  BOTH
FORCE_MAIN_EQUATION  H-W
VARIABLE_STEP        0.75
LENGTHENING_STEP     0
MIN_SURFAREA         12.566
MAX_TRIALS           8
HEAD_TOLERANCE       0.005
SYS_FLOW_TOL         5
LAT_FLOW_TOL         5
MINIMUM_STEP         0.5
THREADS              1


In [25]:
for label, obj in inp1[sections.SUBCATCHMENTS].items():
    print(f'"{label}": {obj},')

"S1": SubCatchment(name='S1', rain_gage='CSI1', outlet='LoopRdStream', area=150.85, imperviousness=50.0, width=2358.6, slope=0.5, curb_length=0.0, snow_pack=nan),
"S2": SubCatchment(name='S2', rain_gage='CSI1', outlet='S1', area=54.267, imperviousness=50.0, width=1198.26, slope=0.5, curb_length=0.0, snow_pack=nan),
"S3": SubCatchment(name='S3', rain_gage='CSI1', outlet='S1', area=17.6, imperviousness=50.0, width=1387.79, slope=0.5, curb_length=0.0, snow_pack=nan),
"S4": SubCatchment(name='S4', rain_gage='CSI1', outlet='S3', area=20.0, imperviousness=50.0, width=754.36, slope=0.5, curb_length=0.0, snow_pack=nan),
"S5": SubCatchment(name='S5', rain_gage='CSI1', outlet='S8', area=14.12, imperviousness=50.0, width=1043.41, slope=0.5, curb_length=0.0, snow_pack=nan),
"S6": SubCatchment(name='S6', rain_gage='CSI1', outlet='Out1', area=24.49, imperviousness=50.0, width=1414.67, slope=0.5, curb_length=0.0, snow_pack=nan),
"S7": SubCatchment(name='S7', rain_gage='CSI1', outlet='Out2', area=13.0

In [26]:
print(inp1[sections.SUBCATCHMENTS].to_inp_lines())

;;     rain_gage        outlet    area imperviousness    width slope curb_length snow_pack
;;name                                                                                    
S1          CSI1  LoopRdStream  150.85             50   2358.6   0.5           0          
S2          CSI1            S1  54.267             50  1198.26   0.5           0          
S3          CSI1            S1    17.6             50  1387.79   0.5           0          
S4          CSI1            S3      20             50   754.36   0.5           0          
S5          CSI1            S8   14.12             50  1043.41   0.5           0          
S6          CSI1          Out1   24.49             50  1414.67   0.5           0          
S7          CSI1          Out2   13.05             50   589.94   0.5           0          
S8          CSI1          Out2   18.45             50   1037.2   0.5           0          
S9          CSI1          OUT3   96.56             50  1234.31   0.5           0          

In [27]:
 inp1[sections.SUBCATCHMENTS].frame

,rain_gage,outlet,area,imperviousness,width,slope,curb_length,snow_pack
name,,,,,,,,
S1,CSI1,LoopRdStream,150.850,50.0,2358.60,0.5,0.0,NaN
S2,CSI1,S1,54.267,50.0,1198.26,0.5,0.0,NaN
S3,CSI1,S1,17.600,50.0,1387.79,0.5,0.0,NaN
S4,CSI1,S3,20.000,50.0,754.36,0.5,0.0,NaN
S5,CSI1,S8,14.120,50.0,1043.41,0.5,0.0,NaN
S6,CSI1,Out1,24.490,50.0,1414.67,0.5,0.0,NaN
S7,CSI1,Out2,13.050,50.0,589.94,0.5,0.0,NaN
S8,CSI1,Out2,18.450,50.0,1037.20,0.5,0.0,NaN
S9,CSI1,OUT3,96.560,50.0,1234.31,0.5,0.0,NaN


In [28]:
print(inp1[sections.VERTICES].to_inp_lines())

;; No Data


In [29]:
print(inp1[sections.POLYGONS].to_inp_lines())

S1  -74.1462460792 40.6060682704
S1  -74.1437952344 40.6054496106
S1  -74.1429862176 40.6055209944
S1  -74.1431527799 40.6037839879
S1  -74.1404877836 40.6029749711
S1  -74.1401784536 40.6006193047
S1  -74.1508622336 40.6004051532
S1  -74.1510050012 40.6029987657
S1  -74.1524802670 40.6028797927
S1  -74.1529799539 40.6039267555
S1  -74.1543362466 40.6044026477
S1  -74.1542886574 40.6073769740
S1  -74.1542886574 40.6074007686
S1  -74.1525754455 40.6074721524
S1  -74.1522185263 40.6070438494
S1  -74.1497914761 40.6067345195
S1  -74.1462460792 40.6060682704
S2  -74.1401333209 40.6006157990
S2  -74.1400333835 40.5992595062
S2  -74.1400333835 40.5988169264
S2  -74.1401618744 40.5983315164
S2  -74.1406615613 40.5976890619
S2  -74.1418608096 40.5961043409
S2  -74.1457155364 40.5961043409
S2  -74.1458297506 40.5987598194
S2  -74.1458583041 40.6005015848
S2  -74.1458154738 40.6004873081
S2  -74.1401333209 40.6006157990
S3  -74.1508477580 40.6004197907
S3  -74.1508733926 40.6004195478
S3  -74.15

In [30]:
#   Print Coordinates of Loop Rd Streem
print(inp1[sections.COORDINATES].to_inp_lines())

;;                    x         y
;;node                           
LoopRdStream  -74.15341  40.60691
Out1            -74.155    40.603
Out2            -74.155      40.6
Out3            -74.153    40.596


In [31]:
#   Print Coordinates of CSI GateWay (Rain Gauge)
print(inp1[sections.SYMBOLS].to_inp_lines())

;;              x         y
;;gage                     
CSI1    -74.14862  40.60413


In [32]:
print(inp1)


{'TITLE': ';;Project Title/Notes\nCSI Campus\n\n;;____________________________________________________________________________________________________', 'OPTIONS': {   'FLOW_UNITS': 'CFS',
    'INFILTRATION': 'GREEN_AMPT',
    'FLOW_ROUTING': 'KINWAVE',
    'LINK_OFFSETS': 'DEPTH',
    'MIN_SLOPE': 0,
    'ALLOW_PONDING': True,
    'SKIP_STEADY_STATE': False,
    'START_DATE': datetime.date(2023, 1, 22),
    'START_TIME': datetime.time(0, 0),
    'REPORT_START_DATE': datetime.date(2023, 1, 22),
    'REPORT_START_TIME': datetime.time(0, 0),
    'END_DATE': datetime.date(2023, 1, 26),
    'END_TIME': datetime.time(23, 59),
    'SWEEP_START': '01/01',
    'SWEEP_END': '12/31',
    'DRY_DAYS': 1,
    'REPORT_STEP': datetime.time(0, 5),
    'WET_STEP': datetime.time(0, 5),
    'DRY_STEP': datetime.time(1, 0),
    'ROUTING_STEP': '0:00:20',
    'RULE_STEP': datetime.time(0, 0),
    'INERTIAL_DAMPING': 'PARTIAL',
    'NORMAL_FLOW_LIMITED': 'BOTH',
    'FORCE_MAIN_EQUATION': 'H-W',
    'VARIAB

In [33]:

#
#   Write inp1
#
Output1InpFile='Out1'+InputInpFile
inp.write_file(Output1InpFile)

'Out1AllCampus4_3V1a.inp'

In [103]:
#
#   This is redundent it is re-reading the original file.
#

In [104]:
inp2 = read_inp_file(InputInpFile)

In [105]:
polygons2=inp2[sections.POLYGONS].frame

In [106]:
polygons2

,polygon
subcatchment,
WholeCampus,"[[-74.1401828, 40.59834549], [-74.14067181, 40..."


In [107]:
print(inp2[sections.POLYGONS].to_inp_lines())

WholeCampus  -74.1401828000 40.5983454900
WholeCampus  -74.1406718100 40.5977051600
WholeCampus  -74.1454944700 40.5913902000
WholeCampus  -74.1456695900 40.5899743000
WholeCampus  -74.1497168400 40.5923967600
WholeCampus  -74.1521482400 40.5936298500
WholeCampus  -74.1527520900 40.5931450600
WholeCampus  -74.1547320900 40.5943031500
WholeCampus  -74.1542587100 40.5948869300
WholeCampus  -74.1558207700 40.5957184300
WholeCampus  -74.1561898500 40.6000337700
WholeCampus  -74.1553097400 40.6037245200
WholeCampus  -74.1544864200 40.6067622900
WholeCampus  -74.1542876900 40.6073584800
WholeCampus  -74.1535495400 40.6074720500
WholeCampus  -74.1538050600 40.6088063900
WholeCampus  -74.1530669100 40.6090051300
WholeCampus  -74.1523571500 40.6071029700
WholeCampus  -74.1510205700 40.6069413200
WholeCampus  -74.1499161600 40.6067810900
WholeCampus  -74.1490942000 40.6066659500
WholeCampus  -74.1475107000 40.6063628400
WholeCampus  -74.1463179900 40.6061665400
WholeCampus  -74.1447164900 40.605

In [108]:
#  The coordinates are not considered a GIS object.
#  They are a Geometry object which as best I can tell are
#  used for the GUI locating and label location on the GUI
#
print(inp2[sections.COORDINATES].to_inp_lines())

;;                  x       y
;;node                       
LoopRdStream  -74.153  40.607


In [109]:
#  After setting the values to 8 decimals it prints the coordinates with 8 decimals which is
#  verified by looking at New_inputfilke2 with an editor.
inp2.write_file('Out2'+OutputInpFile)

'Out2OutCampusInputFile.inp'

In [110]:
 inp2 = SwmmInput.read_file('Out2'+OutputInpFile)

In [111]:
# Unfortunately when reading the new file which has the higher precision, it does not seem to keep it.
# There is an interesting rounding taht was done for x but not y coordinate.
print(inp2[sections.COORDINATES].to_inp_lines())

;;                  x       y
;;node                       
LoopRdStream  -74.153  40.607


In [112]:
#  Polygon precision is fine which is critical.
print(inp2[sections.POLYGONS].to_inp_lines())

WholeCampus  -74.1401828000 40.5983454900
WholeCampus  -74.1406718100 40.5977051600
WholeCampus  -74.1454944700 40.5913902000
WholeCampus  -74.1456695900 40.5899743000
WholeCampus  -74.1497168400 40.5923967600
WholeCampus  -74.1521482400 40.5936298500
WholeCampus  -74.1527520900 40.5931450600
WholeCampus  -74.1547320900 40.5943031500
WholeCampus  -74.1542587100 40.5948869300
WholeCampus  -74.1558207700 40.5957184300
WholeCampus  -74.1561898500 40.6000337700
WholeCampus  -74.1553097400 40.6037245200
WholeCampus  -74.1544864200 40.6067622900
WholeCampus  -74.1542876900 40.6073584800
WholeCampus  -74.1535495400 40.6074720500
WholeCampus  -74.1538050600 40.6088063900
WholeCampus  -74.1530669100 40.6090051300
WholeCampus  -74.1523571500 40.6071029700
WholeCampus  -74.1510205700 40.6069413200
WholeCampus  -74.1499161600 40.6067810900
WholeCampus  -74.1490942000 40.6066659500
WholeCampus  -74.1475107000 40.6063628400
WholeCampus  -74.1463179900 40.6061665400
WholeCampus  -74.1447164900 40.605

In [89]:
#
#
#
#  End Here for Now!
#
#
#

In [ ]:
#########################################################################################################
#
#   Reference Below
#

In [53]:
#   Append and delete test 
############################
#
#   Append items to list
#
NewPolygon=(.123,.456)
inp.POLYGONS['S2'].polygon.append(NewPolygon)
lengthVerticesA = len(inp.POLYGONS['S2'].polygon)
print(lengthVerticesA,' new length')
print(inp.POLYGONS['S2'].polygon[lengthVerticesA-1], 'new vertice')
print('before delete and after append \n')
print(inp.POLYGONS['S2'].polygon)
#
#   End Append section
#
#    Delete additional vertices
#
print(  'deleting \n', LVertices, lengthVerticesA )
del inp.POLYGONS['S2'].polygon[(lengthVerticesA-1)]
print(' after delete')
print(inp.POLYGONS['S2'].polygon)

12  new length
(0.123, 0.456) new vertice
before delete and after append 

[[Decimal('-74.14013332090'), Decimal('40.60061579900')], [Decimal('-74.14003338350'), Decimal('40.59925950620')], [Decimal('-74.14003338350'), Decimal('40.59881692640')], [Decimal('-74.14016187440'), Decimal('40.59833151640')], [Decimal('-74.14066156130'), Decimal('40.59768906190')], [Decimal('-74.14186080960'), Decimal('40.59610434090')], [Decimal('-74.14571553640'), Decimal('40.59610434090')], [Decimal('-74.14582975060'), Decimal('40.59875981940')], [Decimal('-74.14585830410'), Decimal('40.60050158480')], [Decimal('-74.14581547380'), Decimal('40.60048730810')], [Decimal('-74.14013332090'), Decimal('40.60061579900')], (0.123, 0.456)]
deleting 
 [17, 11, 9, 9] 12
 after delete
[[Decimal('-74.14013332090'), Decimal('40.60061579900')], [Decimal('-74.14003338350'), Decimal('40.59925950620')], [Decimal('-74.14003338350'), Decimal('40.59881692640')], [Decimal('-74.14016187440'), Decimal('40.59833151640')], [Decimal(

In [37]:
#   For Reference
#Read starting input INP file and show examples of different statement for reading INP files
# from swmm_api import read_inp_file, SwmmInput
#inp = read_inp_file(r'C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\Site_Drainage_Model.inp')  # type: swmm_api.SwmmInput
# or 
#inp = SwmmInput.read_file('inputfile.inp')
# or
#inp = SwmmInput('inputfile.inp')

In [ ]:
#   Show example of reading and maniputating time series.
from swmm_api.input_file.section_labels import TIMESERIES
#  this still has a problem
# getting a specific section of the inp-file

sec_timeseries = inp[TIMESERIES]  # type: swmm_api.input_file.helpers.InpSection
# or
sec_timeseries = inp.TIMESERIES  # type: swmm_api.input_file.helpers.InpSection

# getting a specific timeseries as pandas.Series
#ts = inp[TIMESERIES]['regenseries'].pandas  # type: pandas.Series
#  MEK this needs to be debugged.

In [39]:
#   Show example of reading and maniputating Junctions.
from swmm_api.input_file.section_labels import JUNCTIONS

# Show example of setting the elevation of a specific node to a new value
# and then resetting the original value.
#save=inp[JUNCTIONS]['J1'].elevation
# 
#inp.JUNCTIONS['J1'].elevation = 210
# 
#inp.JUNCTIONS['J1']['elevation'] = save

In [41]:

#print(str(length)+' length \n') 
# Iterating the index
# same as 'for i in range(len(list))'
#print (inp.POLYGONS.__class__)
#print (a.__class__.__name__)
#for key, value in inp['POLYGONS'].items():
#    print(f'"{key}": "{type(value)}"')   
#for ijrow in len(inp.POLYGONS):
#    print(inp[POLYGONS['S1']].polygon[ijrow][0])
#print(dir(inp.POLYGONS['S1']))
#print((inp.POLYGONS['S1'].__iter__))
#print((inp.POLYGONS['S1'].polygon.__iter__))
#print(len(inp.POLYGONS['S2'].polygon))

#lengthVertices = len(inp.POLYGONS['S1'].polygon)
#print('S1 '+ str(lengthVertices) + '\n')
#print('S1 vertices '+str(LenSubcatchmentS1Rows)+'\n')

#for irow in range(lengthVertices):
#    print(inp.POLYGONS['S1'].polygon[irow])
    

In [56]:
#
#   Assign values to the POLYGONS vertices
#
for irow in range(LenSubcatchmentS1Rows-1):
    #print('POLYGONS.x and y = '+str(inp.POLYGONS['WholeCampus'].polygon[irow][0]),str(inp.POLYGONS['WholeCampus'].polygon[irow][1]))
    #print(' From Text File x,y'+ str(SubcatchmentRows[irow][2]),str(SubcatchmentRows[irow][3]))
    #inp.POLYGONS['WholeCampus'].polygon[irow][0]=Decimal(str(SubcatchmentRows[irow][2])).quantize(Decimal('.00000000001'))
    #inp.POLYGONS['WholeCampus'].polygon[irow][1]=Decimal(str(SubcatchmentRows[irow][3])).quantize(Decimal('.00000000001'))
    inp.POLYGONS['S1'].polygon[irow][0]=Decimal(str(SubcatchmentS1Rows[irow][2])).quantize(Decimal('.00000000001'))
    inp.POLYGONS['S1'].polygon[irow][1]=Decimal(str(SubcatchmentS1Rows[irow][3])).quantize(Decimal('.00000000001'))
print('Revised Polygon vertices')
for irow in range(LenSubcatchmentS1Rows-1):
    print(str(inp.POLYGONS['S1'].polygon[irow][0])+', '+str(inp.POLYGONS['S1'].polygon[irow][1]))

Revised Polygon vertices
-74.14624607920, 40.60606827040
-74.14379523440, 40.60544961060
-74.14298621760, 40.60552099440
-74.14315277990, 40.60378398790
-74.14048778360, 40.60297497110
-74.14017845360, 40.60061930470
-74.15086223360, 40.60040515320
-74.15100500120, 40.60299876570
-74.15248026700, 40.60287979270
-74.15297995390, 40.60392675550
-74.15433624660, 40.60440264770
-74.15428865740, 40.60737697400
-74.15428865740, 40.60740076860
-74.15257544550, 40.60747215240
-74.15221852630, 40.60704384940
-74.14979147610, 40.60673451950
-74.14624607920, 40.60606827040


In [57]:
#
#   Assign values to the POLYGONS vertices S2
#
###   Note: At this time the number of rows in inp.POLYGONS[ ' Each S'] should be equal to the
#           the number of rows in the point data file.  If it is larger, it will not crash
#           but there will be extra entries which need to be deleted.
#           If there are more rows than entries in inp.POLYGONS it will crash.  Some programming needs to be done that 
#           delets the extra rows in inp.POLYGONS and append additional rows if the rows in the data are larger than
#           then inp.POLYGONS.
#
#for irow in range(LenSubcatchmentS2Rows-1):
for irow in range(LenSubcatchmentS2Rows-1):
    #print('POLYGONS.x and y = '+str(inp.POLYGONS['WholeCampus'].polygon[irow][0]),str(inp.POLYGONS['WholeCampus'].polygon[irow][1]))
    #print(' From Text File x,y'+ str(SubcatchmentRows[irow][2]),str(SubcatchmentRows[irow][3]))
    #inp.POLYGONS['WholeCampus'].polygon[irow][0]=Decimal(str(SubcatchmentRows[irow][2])).quantize(Decimal('.00000000001'))
    #inp.POLYGONS['WholeCampus'].polygon[irow][1]=Decimal(str(SubcatchmentRows[irow][3])).quantize(Decimal('.00000000001'))
    inp.POLYGONS['S2'].polygon[irow][0]=Decimal(str(SubcatchmentS2Rows[irow][2])).quantize(Decimal('.00000000001'))
    inp.POLYGONS['S2'].polygon[irow][1]=Decimal(str(SubcatchmentS2Rows[irow][3])).quantize(Decimal('.00000000001'))
print('Revised Polygon vertices')
for irow in range(LenSubcatchmentS2Rows-1):
    print(str(inp.POLYGONS['S2'].polygon[irow][0])+', '+str(inp.POLYGONS['S2'].polygon[irow][1]))

Revised Polygon vertices
-74.14013332090, 40.60061579900
-74.14003338350, 40.59925950620
-74.14003338350, 40.59881692640
-74.14016187440, 40.59833151640
-74.14066156130, 40.59768906190
-74.14186080960, 40.59610434090
-74.14571553640, 40.59610434090
-74.14582975060, 40.59875981940
-74.14585830410, 40.60050158480
-74.14581547380, 40.60048730810
-74.14013332090, 40.60061579900


In [ ]:
#   Reference Below
#
#   Read text files with Coordinates of Loop Road Stream Sensor
#
#   Define File Name, Open file, Read file header and data.
#
# initializing the titles and rows list
#
fields = []
StreamSensorRows = []
# csv file name
filename = "CampusStreamSensor.txt"
# opening the CSV file
with open(filename, mode ='r')as file:

# reading the CSV file
    csvFile = csv.reader(file)
    print('\n fields \n')
    fields = next(csvFile)
    print(fields)
    print(' end fields \n')
    for row in csvFile:
        StreamSensorRows.append(row)
        print (row,'\n')
        for col in row:
            print("%10s"%col,end=" "),
        print('\n')
        # get total number of rows
    print("Total no. of rows: %d"%(csvFile.line_num))
    print(StreamSensorRows[0:csvFile.line_num],' end rows \n')
print(StreamSensorRows[0][2], ' \n ' )
print(StreamSensorRows[0][3], ' \n ' )
      
print(' end print col and row \n')
#
#   Read Rain Gague location.  And assign to Gage  
#
#   Define File Name, Open file, Read file header and data.
#
# initializing the titles and rows list
#
fields = []
RainGaugeRows = []
# csv file name
filename = "CSIGateWay.txt"
# opening the CSV file
with open(filename, mode ='r')as file:
# reading the CSV file
    csvFile = csv.reader(file)
    print('\n fields \n')
    fields = next(csvFile)
    print(fields)
    print(' end fields \n')
    for row in csvFile:
        RainGaugeRows.append(row)
        print (row,'\n')
        for col in row:
            print("%10s"%col,end=" "),
        print('\n')
        # get total number of rows
    print("Total no. of rows: %d"%(csvFile.line_num))
    print(RainGaugeRows[0:csvFile.line_num],' \n end rows \n')  
print(' end print Rain Gauge col and row \n')

In [112]:
#   This section shows the use of GeoSeries which does not seem to solve the precision problem.
inp2.write_file('Out3'+OutputInpFile)
inp3 = SwmmInput.read_file('Out3'+OutputInpFile)

coords = inp.COORDINATES.geo_series  # type: geoandas.GeoSeries with points for all nodes
#vertices = inp.VERTICES.geo_series  # type: geoandas.GeoSeries with lines for all links
polygons = inp.POLYGONS.geo_series  # type: geoandas.GeoSeries with polygons for all subcatchments

# create geopackage of all objects in inp file
write_geo_package(inp, gpkg_fn=r'C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\Campus\geopackage.gpkg', driver='GPKG', label_sep='.', crs="EPSG:32633")

# read above written geopackage and convert it to inp-data
inp_new = gpkg_to_swmm(r'C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\Campus\geopackage.gpkg', label_sep='.')
inp_new.write_file(r'C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\Campus\New_inputfileGEO.inp')
print(coords)
from swmm_api import CONFIG
print(CONFIG)

JUNCTIONS | OUTFALLS | DIVIDERS | STORAGE | CONDUITS | ORIFICES | OUTLETS | PUMPS | WEIRS | SUBCATCHMENTS
    -     |   0.1s   |    -     |    -    |    -     |    -     |    -    |   -   |   -   |     0.0s     


AttributeError: 'NoneType' object has no attribute 'keys'

In [57]:
inp4 = SwmmInput.read_file(r'C:\Zhanyang\SensorData\CompoundFlooding\SWMM\PySWMM\New_inputfileGEO.inp')

In [58]:
print("INP4, POLYGONS")
print(inp4[sections.POLYGONS].to_inp_lines())
print("INP4, COORDINATES")
print(inp4[sections.COORDINATES].to_inp_lines())
print("INP4, VERTICES")
print(inp4[sections.VERTICES].to_inp_lines())

INP4, POLYGONS
S1  282.65700000 1334.81000000
S1  111.70000000 1101.60400000
S1  172.52500000 1062.74300000
S1  231.66000000 1027.26200000
S1  306.00200000 990.09200000
S1  370.20600000 959.67900000
S1  409.06600000 946.16300000
S1  444.54700000 936.02500000
S1  493.54500000 924.19800000
S1  532.40500000 915.75000000
S1  569.57600000 907.30200000
S1  610.12500000 897.16500000
S1  655.74400000 897.16500000
S1  684.33800000 1318.70000000
S1  651.04300000 1321.92200000
S1  596.26900000 1332.66200000
S1  551.16000000 1346.62400000
S1  495.31200000 1367.03000000
S1  455.57300000 1384.21400000
S1  410.46500000 1409.99100000
S1  386.83600000 1427.17500000
S1  363.20800000 1442.21100000
S1  282.65700000 1334.81000000
S2  678.96700000 1238.14900000
S2  673.58400000 1152.90300000
S2  655.74400000 897.16500000
S2  758.80800000 893.78600000
S2  817.94300000 895.47500000
S2  880.45800000 898.85500000
S2  921.00700000 905.61300000
S2  978.45300000 920.81900000
S2  1042.65700000 937.71500000
S2  1103

In [14]:
#
#   Read text files with Coordinates of Loop Road Stream Sensor
#
#   Define File Name, Open file, Read file header and data.
#
import csv
#
# initializing the titles and rows list
#
fields = []
rows = []
# csv file name
filename = "CampusStreamSensor.txt"
# opening the CSV file
with open(filename, mode ='r')as file:

# reading the CSV file
    csvFile = csv.reader(file)
    print('\n fields \n')
    fields = next(csvFile)
    print(fields)
    print(' end fields \n')
    for row in csvFile:
        rows.append(row)
        print (row,'\n')
        for col in row:
            print("%10s"%col,end=" "),
        print('\n')
        # get total number of rows
    print("Total no. of rows: %d"%(csvFile.line_num))
    print(rows[0:csvFile.line_num],' end rows \n')
print(rows[0][3], ' \n ')

print(' end print col and row \n')
#
#   Assign coordinates to LoopRdStream 
#
inp.COORDINATES['LoopRdStream'].x = rows[0][2]
inp.COORDINATES['LoopRdStream'].y = rows[0][3]
print(inp.COORDINATES['LoopRdStream'].x, ' x \n ')
print(inp.COORDINATES['LoopRdStream'].y, ' y \n ')



 fields 

['FID', 'Name', 'POINT_X', 'POINT_Y', 'Gateway', 'Source']
 end fields 

['0', 'LoopRd Stream', '-74.153414144300001', '40.606906320800000', 'CSI', 'Rain'] 

         0 LoopRd Stream -74.153414144300001 40.606906320800000        CSI       Rain 

Total no. of rows: 2
[['0', 'LoopRd Stream', '-74.153414144300001', '40.606906320800000', 'CSI', 'Rain']]  end rows 

40.606906320800000  
 
 end print col and row 

-74.153414144300001  x 
 
40.606906320800000  y 
 


In [14]:
#
#   Read text files with Coordinates of Loop Road Stream Sensor
#
#   Define File Name, Open file, Read file header and data.
#
import csv
#
# initializing the titles and rows list
#
fields = []
rows = []
# csv file name
filename = "CampusStreamSensor.txt"
# opening the CSV file
with open(filename, mode ='r')as file:

# reading the CSV file
    csvFile = csv.reader(file)
    print('\n fields \n')
    fields = next(csvFile)
    print(fields)
    print(' end fields \n')
    for row in csvFile:
        rows.append(row)
        print (row,'\n')
        for col in row:
            print("%10s"%col,end=" "),
        print('\n')
        # get total number of rows
    print("Total no. of rows: %d"%(csvFile.line_num))
    print(rows[0:csvFile.line_num],' end rows \n')
print(rows[0][3], ' \n ')

print(' end print col and row \n')
#
#   Assign coordinates to LoopRdStream 
#
inp.COORDINATES['LoopRdStream'].x = rows[0][2]
inp.COORDINATES['LoopRdStream'].y = rows[0][3]
print(inp.COORDINATES['LoopRdStream'].x, ' x \n ')
print(inp.COORDINATES['LoopRdStream'].y, ' y \n ')



 fields 

['FID', 'Name', 'POINT_X', 'POINT_Y', 'Gateway', 'Source']
 end fields 

['0', 'LoopRd Stream', '-74.153414144300001', '40.606906320800000', 'CSI', 'Rain'] 

         0 LoopRd Stream -74.153414144300001 40.606906320800000        CSI       Rain 

Total no. of rows: 2
[['0', 'LoopRd Stream', '-74.153414144300001', '40.606906320800000', 'CSI', 'Rain']]  end rows 

40.606906320800000  
 
 end print col and row 

-74.153414144300001  x 
 
40.606906320800000  y 
 


In [15]:
import pandas
 
# reading the CSV file
csvFile = pandas.read_csv('CampusStreamSensor.txt')
 
# displaying the contents of the CSV file
print(csvFile)

   FID           Name    POINT_X    POINT_Y Gateway Source
0    0  LoopRd Stream -74.153414  40.606906     CSI   Rain
